In [1]:
import torch
from torch import nn
from torchvision.transforms import ToTensor
import numpy as np
from PIL import Image, ImageDraw, ImageCms
import os
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interactive, VBox, interact, fixed
import ipywidgets
import matplotlib.pyplot as plt
from io import BytesIO
import napari
from napari.utils.translations import trans
import subprocess

In [2]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
def create_2d_grid(x_range, y_range, resolution):
    """
    Create a 2D grid with specified x and y ranges and resolution.

    Parameters:
    x_range (tuple): (min_x, max_x)
    y_range (tuple): (min_y, max_y)
    resolution (tuple): (x_resolution, y_resolution)

    Returns:
    np.ndarray: 2D grid as an (N, 2) array where each row is an [x, y] pair.
    """
    x_min, x_max = x_range
    y_min, y_max = y_range
    x_res, y_res = resolution

    x_values = np.linspace(x_min, x_max, x_res)
    y_values = np.linspace(y_min, y_max, y_res)
    
    x_grid, y_grid = np.meshgrid(x_values, y_values)
    
    grid = np.vstack([x_grid.ravel(), y_grid.ravel()]).T
    return grid

def create_3d_grid(x_range, y_range, z_range, resolution, alpha):
    """
    Create a 2D grid with specified x and y ranges and resolution.

    Parameters:
    x_range (tuple): (min_x, max_x)
    y_range (tuple): (min_y, max_y)
    resolution (tuple): (x_resolution, y_resolution)

    Returns:
    np.ndarray: 2D grid as an (N, 2) array where each row is an [x, y] pair.
    """
    
    x_res, y_res, z_res = resolution

    x_min, x_max = (alpha*x_res/2, -alpha*x_res/2)
    y_min, y_max = (alpha*y_res/2, -alpha*y_res/2)
    z_min, z_max = (alpha*z_res/2, -alpha*z_res/2)

    x_values = np.linspace(x_min, x_max, x_res)
    y_values = np.linspace(y_min, y_max, y_res)
    z_values = np.linspace(z_min, z_max, z_res)

    x_grid, y_grid, z_grid = np.meshgrid(x_values, y_values, z_values)
    
    grid = np.vstack([x_grid.ravel(), y_grid.ravel(), z_grid.ravel()]).T
    return grid

In [4]:
class GrayScaleModel(nn.Module):
    def __init__(self, min_w=-0.5, max_w=0.5, N_neurons=50, N_layers=50, seed = 100):
        super().__init__()
        # self.flatten = nn.Flatten()

        self.linear_relu_stack = nn.Sequential()


        lin_layer = nn.Linear(2, N_neurons)
        torch.nn.init.uniform_(lin_layer.weight, min_w, max_w)
        
        self.linear_relu_stack.append( lin_layer )
        self.linear_relu_stack.append(nn.Tanh()),
        
        for _ in range(N_layers):

            lin_layer = nn.Linear(N_neurons, N_neurons)
            torch.nn.init.uniform_(lin_layer.weight, min_w, max_w)
            
            self.linear_relu_stack.append( lin_layer  )
            self.linear_relu_stack.append(nn.Tanh()),


        lin_layer = nn.Linear(N_neurons, 1)
        torch.nn.init.uniform_(lin_layer.weight, min_w, max_w)
        
        self.linear_relu_stack.append( lin_layer  )
        self.linear_relu_stack.append(nn.Tanh())


    def forward(self, x):
        # x = self.flatten(x)
        with torch.no_grad():
            logits = self.linear_relu_stack(x)
            return logits


class RGBScaleModel(nn.Module):
    def __init__(self, min_w=-0.5, max_w=0.5, N_neurons=50, N_layers=50, seed = 100):
        super().__init__()
        # self.flatten = nn.Flatten()

        self.linear_relu_stack = nn.Sequential()


        lin_layer = nn.Linear(2, N_neurons)
        torch.nn.init.uniform_(lin_layer.weight, min_w, max_w)
        
        self.linear_relu_stack.append( lin_layer )
        self.linear_relu_stack.append(nn.Tanh()),
        
        for _ in range(N_layers):

            lin_layer = nn.Linear(N_neurons, N_neurons)
            torch.nn.init.uniform_(lin_layer.weight, min_w, max_w)
            
            self.linear_relu_stack.append( lin_layer  )
            self.linear_relu_stack.append(nn.Tanh()),


        lin_layer = nn.Linear(N_neurons, 3)
        torch.nn.init.uniform_(lin_layer.weight, min_w, max_w)
        
        self.linear_relu_stack.append( lin_layer  )
        self.linear_relu_stack.append(nn.Tanh())


    def forward(self, x):
        # x = self.flatten(x)
        with torch.no_grad():
            logits = self.linear_relu_stack(x)
            return logits

class RGBAScaleModel(nn.Module):
    def __init__(self, min_w=-0.5, max_w=0.5, N_neurons=50, N_layers=50, seed = 100):
        super().__init__()
        # self.flatten = nn.Flatten()

        self.linear_relu_stack = nn.Sequential()


        lin_layer = nn.Linear(2, N_neurons)
        torch.nn.init.uniform_(lin_layer.weight, min_w, max_w)
        
        self.linear_relu_stack.append( lin_layer )
        self.linear_relu_stack.append(nn.Tanh()),
        
        for _ in range(N_layers):

            lin_layer = nn.Linear(N_neurons, N_neurons)
            torch.nn.init.uniform_(lin_layer.weight, min_w, max_w)
            
            self.linear_relu_stack.append( lin_layer  )
            self.linear_relu_stack.append(nn.Tanh()),


        lin_layer = nn.Linear(N_neurons, 4)
        torch.nn.init.uniform_(lin_layer.weight, min_w, max_w)
        
        self.linear_relu_stack.append( lin_layer  )
        self.linear_relu_stack.append(nn.Tanh())


    def forward(self, x):
        # x = self.flatten(x)
        with torch.no_grad():
            logits = self.linear_relu_stack(x)
            return logits

class CMYKModel(nn.Module):
    def __init__(self, input_size = 2, min_w=-0.5, max_w=0.5, N_neurons=50, N_layers=50, seed = 100):
        super().__init__()
        # self.flatten = nn.Flatten()

        self.linear_relu_stack = nn.Sequential()
        
        lin_layer = nn.Linear(input_size, N_neurons, dtype=torch.float, device=device)
        torch.nn.init.uniform_(lin_layer.weight, min_w, max_w)
        
        self.linear_relu_stack.append( lin_layer )
        self.linear_relu_stack.append(nn.Tanh()),
        
        for _ in range(N_layers):

            lin_layer = nn.Linear(N_neurons, N_neurons, dtype=torch.float, device=device)
            torch.nn.init.uniform_(lin_layer.weight, min_w, max_w)
            
            self.linear_relu_stack.append( lin_layer  )
            self.linear_relu_stack.append(nn.Tanh()),


        lin_layer = nn.Linear(N_neurons, 4, dtype=torch.float, device=device)
        torch.nn.init.uniform_(lin_layer.weight, min_w, max_w)
        
        self.linear_relu_stack.append( lin_layer  )
        self.linear_relu_stack.append(nn.Tanh())


    def forward(self, x):
        # x = self.flatten(x)
        with torch.no_grad():
            logits = self.linear_relu_stack(x)
            return logits
        

In [5]:
# img = (img - img.min()) / (img.max() - img.min())*225
# img = np.uint32(img)
# img = Image.fromarray(img)
# img = img.convert('RGBA')
# img.save(f"very_big.png")   

In [6]:
def get_rgb_image(x, model, x_shape, y_shape):

    x = torch.tensor(x, device=device)
    x = x.to(torch.float32)
    y = model.forward(x).cpu()
    img = y.reshape(y_shape,x_shape,3)
    img = (img - img.min()) / (img.max() - img.min())*225
    img = np.uint8(img)
    img = Image.fromarray(img, mode='HSV')
    img = img.convert('RGB')

    return img

def get_rgba_image(x, model, x_shape, y_shape):

    x = torch.tensor(x, device=device)
    x = x.to(torch.float32)
    y = model.forward(x).cpu()
    img = y.reshape(y_shape,x_shape,4)
    img = (img - img.min()) / (img.max() - img.min())*225
    img = np.uint8(img)
    img = Image.fromarray(img, mode='RGBA')

    return img


def get_cmyk_image(x, model, x_shape, y_shape):

    x = torch.tensor(x, device=device)
    x = x.to(torch.float32)
    y = model.forward(x).cpu()
    img = y.reshape(y_shape,x_shape,4)
    img = (img - img.min()) / (img.max() - img.min())*225
    img = np.uint8(img)
    img = Image.fromarray(img, mode='CMYK')
    # img = img.convert('RGB')
    # img = img.convert('CMYK')

    return img


def get_gray_image(x, model, x_shape, y_shape):

    x = torch.tensor(x, device=device)
    x = x.to(torch.float32)
    y = model.forward(x).cpu()
    img = y.reshape(y_shape,x_shape)
    img = (img - img.min()) / (img.max() - img.min())*225
    img = np.uint8(img)
    img = Image.fromarray(img, mode='L')
    img = img.convert('RGB')

    return img

def rotate_3d_tensor(tensor, angle_x=0.0, angle_y=0.0, angle_z=0.0):
    """
    Apply a 3D rotation to a PyTorch tensor using rotation matrices.

    Parameters:
    - tensor: torch.Tensor of shape (N, 3) representing N points in 3D space.
    - angle_x: Rotation angle around the X-axis in radians.
    - angle_y: Rotation angle around the Y-axis in radians.
    - angle_z: Rotation angle around the Z-axis in radians.

    Returns:
    - Rotated tensor of shape (N, 3).
    """


    # Rotation matrix around X-axis
    Rx = torch.tensor(np.array([
        [1, 0, 0],
        [0, np.cos(angle_x), -np.sin(angle_x)],
        [0, np.sin(angle_x), np.cos(angle_x)]
    ]), dtype=tensor.dtype, device=tensor.device)

    # Rotation matrix around Y-axis
    Ry = torch.tensor(np.array([
        [np.cos(angle_y), 0, np.sin(angle_y)],
        [0, 1, 0],
        [-np.sin(angle_y), 0, np.cos(angle_y)]
    ]), dtype=tensor.dtype, device=tensor.device)

    # Rotation matrix around Z-axis
    Rz = torch.tensor(np.array([
        [np.cos(angle_z), -np.sin(angle_z), 0],
        [np.sin(angle_z), np.cos(angle_z), 0],
        [0, 0, 1]
    ]), dtype=tensor.dtype, device=tensor.device)

    # Compute the combined rotation matrix
    R = Rz @ Ry @ Rx  # Apply rotations in order: X -> Y -> Z

    # Apply the rotation
    rotated_tensor = tensor @ R.T  # Transpose because torch.matmul assumes row vectors

    return rotated_tensor



def translate_3d_tensor(tensor, vec3):
    """
    Apply a 3D rotation to a PyTorch tensor using rotation matrices.

    Parameters:
    - tensor: torch.Tensor of shape (N, 3) representing N points in 3D space.
    - angle_x: Rotation angle around the X-axis in radians.
    - angle_y: Rotation angle around the Y-axis in radians.
    - angle_z: Rotation angle around the Z-axis in radians.

    Returns:
    - Rotated tensor of shape (N, 3).
    """

    tensor = tensor + vec3
    return tensor


def convert_cmyk_to_rgb(img):

    cmyk_profile_path = "CMYK_profiles/USWebUncoated.icc"  # CMYK profile
    rgb_profile_path = "rgb_profiles/sRGB Color Space Profile.icm"  # RGB profile

    # Build the transform from CMYK to RGB using ICC profiles
    cmyk_to_rgb_transform = ImageCms.buildTransform(cmyk_profile_path, rgb_profile_path, 'CMYK', 'RGB')

    # Apply the ICC profile conversion
    rgb_image = ImageCms.applyTransform(img, cmyk_to_rgb_transform)

    # Save the converted image
    return rgb_image


def get_noise(model):
    model_size = len(model.linear_relu_stack)
    noises = []

    for i in range(0,model_size,2):
        layer = model.linear_relu_stack[i]
        shape = layer.weight.shape

        noise = torch.zeros(shape, device=device, dtype=torch.float32)
        noise.uniform_(-0.5,0.5)

        noises.append(noise)
    
    return noises

def apply_noise(model, noise_weight, noises=None):
    model_size = len(model.linear_relu_stack)
    

    if noises is None:
        
        noises = []
        for i in range(0,model_size,2):
            layer = model.linear_relu_stack[i]
            shape = layer.weight.shape

            noise = torch.zeros(shape, device=device, dtype=torch.float32)
            noise.uniform_(-0.5,0.5)

            noises.append(noise)
    
    for i in range(0,model_size,2):

        layer = model.linear_relu_stack[i]
        layer.weight.data = layer.weight.data + noises[i//2] * noise_weight


def scale_paramns(model, noise, noise_weight):
    model_size = len(model.linear_relu_stack)

    for i in range(0,model_size,2):

        layer = model.linear_relu_stack[i]
        layer.weight.data = layer.weight.data * noise_weight

## Fixed Noise

In [34]:
#### Fixed noise

# A# aspect raio 1:1.414

aspect = (1,1.414)

c = 0.5/1.414

k = 4000
W = 1*k
H = int(1.414*k)

ly = 0.5
lx = c*0.5
x_range = (-lx, lx)
y_range = (-ly, ly)
resolution = (W, H)

neurons = 50
layers = 50


os.makedirs(f"cmyk_images_fixed_noise_{neurons}_{layers}_res_{W}x{H}", exist_ok = True)

# model = RGBScaleModel(N_neurons=neurons, N_layers=layers).to(device)
model = CMYKModel(N_neurons=neurons, N_layers=layers).to(device)
x = create_2d_grid(x_range, y_range, resolution)

images = []
dx = 0.05
model_size = len(model.linear_relu_stack)

noises = []

for i in range(0,model_size,2):
    layer = model.linear_relu_stack[i]
    shape = layer.weight.shape

    noise = torch.zeros(shape, device=device, dtype=torch.float32)
    noise.uniform_(-dx,dx)

    noises.append(noise)



for n in range(20):
  
    for i in range(0,model_size,2):

        layer = model.linear_relu_stack[i]
        layer.weight.data = layer.weight.data + noises[i//2]
    
    image = get_cmyk_image(x, model, x_shape = W, y_shape=H)
    
    prefix = str(n)
    prefix_size = len(prefix)

    filename = "".join(["0" for j in range( 5 - prefix_size )] + [str(n)] )

    print(filename)
    image.save(f"cmyk_images_fixed_noise_{neurons}_{layers}_res_{W}x{H}/{filename}.jpeg")  
            

KeyboardInterrupt: 

## Sine Noise

In [ ]:
#### Sine noise

r = 2000
l = 0.5
x_range = (-l, l)
y_range = (-l, l)
resolution = (r, r)

model = RGBScaleModel(N_neurons=50, N_layers=40).to(device)
x = create_2d_grid(x_range, y_range, resolution)

images = []
dx = 0.001
model_size = len(model.linear_relu_stack)

noises = []

os.makedirs(f"gray_images_random_walk_{neurons}_{layers}_res_{W}x{H}", exist_ok = True)


for i in range(0,model_size,2):
    layer = model.linear_relu_stack[i]
    shape = layer.weight.shape

    noise = torch.zeros(shape, device=device, dtype=torch.float32)
    noise.uniform_(-dx,dx)

    noises.append(noise)

dTheta = 2*np.pi/500
theta = 0

for n in range(500):
  
    for i in range(0,model_size,2):

        layer = model.linear_relu_stack[i]
        layer.weight.data = layer.weight.data + noises[i//2]*np.sin(theta)*3

    theta += dTheta
    
    image = get_rgb_image(x, model)
    
    prefix = str(n)
    prefix_size = len(prefix)

    filename = "".join(["0" for j in range( 5 - prefix_size )] + [str(n)] )

    print(filename)
    image.save(f"images_rgb/{filename}.png")  

## Sine Noise 2

In [ ]:
#### Sine noise2

W = 500
H = 500
l = 1
x_range = (-l, l)
y_range = (-l, l)
resolution = (W, H)

neurons = 30
layers = 40
# model = RGBAScaleModel(N_neurons=neurons, N_layers=layers).to(device)
model = CMYKModel(0,0,N_neurons=neurons, N_layers=layers).to(device)

x = create_2d_grid(x_range, y_range, resolution)

os.makedirs(f"cmyk_images_sine2_{neurons}_{layers}", exist_ok = True)

images = []
dx = 0.01
model_size = len(model.linear_relu_stack)

noises = []
thetas = []

for i in range(0,model_size,2):
    layer = model.linear_relu_stack[i]
    shape = layer.weight.shape

    noise = torch.zeros(shape, device=device, dtype=torch.float32)
    noise.uniform_(-dx,dx)

    noises.append(noise)
    thetas.append( np.random.uniform(0, 2*np.pi))

dTheta = 2*np.pi/500

for n in range(500):
  
    for i in range(0,model_size,2):

        layer = model.linear_relu_stack[i]
        layer.weight.data = layer.weight.data + noises[i//2]*np.sin(thetas[i//2])*1.5
        thetas[i//2] += dTheta
    
    #image = get_rgb_image(x, model, x_shape=W, y_shape=H)
    image = get_cmyk_image(x, model, x_shape=W, y_shape=H)
    
    
    prefix = str(n)
    prefix_size = len(prefix)

    filename = "".join(["0" for j in range( 5 - prefix_size )] + [str(n)] )

    print(filename)
    image.save(f"cmyk_images_sine2_{neurons}_{layers}/{filename}.jpeg")  

## Randon Walk RGB

In [ ]:
#### Random walk RGB

r = 2000
l = 0.5
x_range = (-l, l)
y_range = (-l, l)
resolution = (r, r)

model = RGBScaleModel(N_neurons=50, N_layers=40).to(device)
x = create_2d_grid(x_range, y_range, resolution)

images = []
dx = 0.001
model_size = len(model.linear_relu_stack)

for n in range(500):
  
    for i in range(0,model_size,2):

        layer = model.linear_relu_stack[i]
        shape = layer.weight.shape
    
        noise = torch.zeros(shape, device=device, dtype=torch.float32)
        noise.uniform_(-dx,dx)
        
        layer = model.linear_relu_stack[i]
        layer.weight.data = layer.weight.data + noise
    
    image = get_rgb_image(x, model)
    
    prefix = str(n)
    prefix_size = len(prefix)

    filename = "".join(["0" for j in range( 5 - prefix_size )] + [str(n)] )

    print(filename)
    image.save(f"images_rgb/{filename}.png")  

In [ ]:
1080/1920


## Random Walk Gray

In [ ]:
#### Random walk gary

W = 1920
H = 1080
ly = 0.5
lx = 0.8
x_range = (-lx, lx)
y_range = (-ly, ly)
resolution = (W, H)



neurons = 75
layers = 20
model = GrayScaleModel(N_neurons=neurons, N_layers=layers).to(device)
x = create_2d_grid(x_range, y_range, resolution)

os.makedirs(f"gray_images_random_walk_{neurons}_{layers}_res_{W}x{H}", exist_ok = True)


images = []
dx = 0.001
model_size = len(model.linear_relu_stack)

for n in range(500):
  
    for i in range(0,model_size,2):

        layer = model.linear_relu_stack[i]
        shape = layer.weight.shape
    
        noise = torch.zeros(shape, device=device, dtype=torch.float32)
        noise.uniform_(-dx,dx)
        
        layer = model.linear_relu_stack[i]
        layer.weight.data = layer.weight.data + noise
    
    image = get_gray_image(x, model, x_shape=W, y_shape=H)
    
    prefix = str(n)
    prefix_size = len(prefix)

    filename = "".join(["0" for j in range( 5 - prefix_size )] + [str(n)] )

    print(filename)
    image.save(f"gray_images_random_walk_{neurons}_{layers}_res_{W}x{H}/{filename}.png")  

In [ ]:
r = 1000
l = 0.5
x_range = (-l, l)
y_range = (-l, l)
resolution = (r, r)


x = create_2d_grid(x_range, y_range, resolution)

images = []
dx = 0.001
model_size = len(model.linear_relu_stack)
for N in range(10,100,10):

    for L in range(10,100,10):

        model = RGBScaleModel(N_neurons=N, N_layers=L).to(device)
        image = get_rgb_image(x, model)
        
        filename = f"{N}_{L}.png"

        print(filename)
        image.save(f"space/{filename}.png")  

## XYZ CMYK no noise

In [ ]:
r = 1000
rz = 60
l = 68
x_range = (-l, l)
y_range = (-l, l)
z_range = (-20, 20)
resolution = (r, r, rz)

x = create_3d_grid(x_range, y_range, z_range, resolution)

neurons = 49
layers = 25

images = []

model = CMYKModel(input_size=3, N_neurons=neurons, N_layers=layers).to(device)

x = torch.tensor(x, dtype=torch.float, device=device)
output = model.forward(x).cpu()
output = output.reshape((r,r,rz,4))
output = output[:,:,:,:3]*255

In [ ]:
output = output.to(torch.int8)

In [ ]:
viewer = napari.Viewer(ndisplay=3)
image_layer = viewer.add_image(
    output, rendering='mip', name='volume', blending='additive', opacity=0.25
)

plane_parameters = {
    'position': (32, 32, 32),
    'normal': (0, 1, 0),
    'thickness': 10,
}

plane_layer = viewer.add_image(
    output,
    rendering='average',
    name='plane',
    depiction='plane',
    blending='additive',
    opacity=0.5,
    plane=plane_parameters
)

viewer.axes.visible = True
viewer.camera.angles = (45, 45, 45)
viewer.camera.zoom = 5
viewer.text_overlay.text = trans._(
    """
shift + click and drag to move the plane
press 'x', 'y' or 'z' to orient the plane along that axis around the cursor
press 'o' to orient the plane normal along the camera view direction
press and hold 'o' then click and drag to make the plane normal follow the camera
"""
)
viewer.text_overlay.visible = True
napari.run()


In [ ]:
os.makedirs(f"cmyk_xyz_{neurons}_{layers}", exist_ok = True)

for n in range(rz):

    img = output[:,:,n,:]

    img = (img - img.min()) / (img.max() - img.min())*225
    img = np.uint8(img)
    img = Image.fromarray(img, mode='CMYK')


    prefix = str(n)
    prefix_size = len(prefix)

    filename = "".join(["0" for j in range( 5 - prefix_size )] + [str(n)] )

    print(filename)
    img.save(f"cmyk_xyz_{neurons}_{layers}/{filename}.jpeg")  

00000
00001
00002
00003
00004
00005
00006
00007
00008
00009
00010
00011
00012
00013
00014
00015
00016
00017
00018
00019
00020
00021
00022
00023
00024
00025
00026
00027
00028
00029
00030
00031
00032
00033
00034
00035
00036
00037
00038
00039
00040
00041
00042
00043
00044
00045
00046
00047
00048
00049
00050
00051
00052
00053
00054
00055
00056
00057
00058
00059


## Interactive

In [ ]:
# Placeholder update logic (Modify this to adjust the image based on sliders)
W = 500
H = 500
l = 1
x_range = (-l, l)
y_range = (-l, l)
resolution = (W, H)

neurons = 30
layers = 40
# model = RGBAScaleModel(N_neurons=neurons, N_layers=layers).to(device)
model = CMYKModel(N_neurons=neurons, N_layers=layers).to(device)

x = create_2d_grid(x_range, y_range, resolution)

# os.makedirs(f"cmyk_images_sine2_{neurons}_{layers}", exist_ok = True)

#image = get_rgb_image(x, model, x_shape=W, y_shape=H)
image = get_cmyk_image(x, model, x_shape=W, y_shape=H)

image.show()

In [ ]:
neurons = 40
layers = 40
old_N = 0
old_L = 0
model = CMYKModel(input_size=3, N_neurons=neurons, N_layers=layers).to(device)
# Update function
def update_image( l, z, neurons, layers):
    global old_N, old_L, model
    # Placeholder update logic (Modify this to adjust the image based on sliders)
    W = 500
    H = 500
    x_range = (-l, l)
    y_range = (-l, l)
    resolution = (W, H)

    if (old_N != neurons) or (old_L != layers):
        old_N = neurons
        old_L = layers
        model = CMYKModel(N_neurons=neurons, N_layers=layers).to(device)

    # model = RGBAScaleModel(N_neurons=neurons, N_layers=layers).to(device)
    
    x = create_2d_grid(x_range, y_range, resolution)

    #image = get_rgb_image(x, model, x_shape=W, y_shape=H)
    image = get_cmyk_image(x, model, x_shape=W, y_shape=H)
    
    draw = ImageDraw.Draw(image)
    # draw.rectangle(xy=(foo, bar, essential_value_one, essential_value_two))

    imgBytes = BytesIO()
    image.save(imgBytes, format="jpeg")
    
    return ipywidgets.Image(value=imgBytes.getvalue())



In [ ]:
interact(update_image, l=(0,200,1), neurons=(1,50,1), layers=(1,50,1))

interactive(children=(IntSlider(value=100, description='l', max=200), IntSlider(value=25, description='neurons…

<function __main__.update_image(l, neurons, layers)>

## Zooming

In [220]:
r = 1000
rz = 1
l = 5
x_range = (-l, l)
y_range = (-l, l)
z_range = (0, 0)
Rx = 1920*2
Ry = 1080*2
resolution = (Rx, Ry, rz)

neurons = 30
layers = 40

images = []

# model = CMYKModel(input_size=3, N_neurons=neurons, N_layers=layers).to(device)
# torch.save(model, f"saved_models/curated_{neurons}_{layers}_00066.pt")
model = torch.load(f"saved_models/curated_{neurons}_{layers}_00030.pt").to(device)

step=0.005
x = create_3d_grid(x_range, y_range, z_range, resolution, step)


In [221]:
# noises = get_noise(model)

In [222]:
os.makedirs(f"cmyk_xyz_rotation_{neurons}_{layers}", exist_ok = True)
x_r = torch.tensor(x, dtype=torch.float, device=device)

w = torch.tensor(1, device='cuda', dtype=torch.float)
noise_weight = (0.3/0.001)**(1/1800)
step_weight = (1e-3)**(1/1800)
# translation_step = torch.tensor([0,10,0], dtype=torch.float, device=device)

for n,t in enumerate(np.linspace(0, 2*np.pi, 30*60)):

    # model = torch.load(f"saved_models/{neurons}_{layers}_noise.pt").to(device)
    # step *= step_weight
    w = 2*torch.sin(torch.tensor(t/2))
    # translation_step += torch.tensor([0,0.1,0], dtype=torch.float, device=device)
    x_r = create_3d_grid(x_range, y_range, z_range, resolution, step)
    x_r = torch.tensor(x_r, dtype=torch.float, device=device)
    x_r = rotate_3d_tensor(x_r, 0, t, t*0.1)
    # x_r = translate_3d_tensor(x_r,translation_step)
    # apply_noise(model, w, noises)
    output = model.forward(x_r).cpu()
    output = output.reshape((Ry,Rx,rz,4))

    img = output[:,:,:]

    img = (img - img.min()) / (img.max() - img.min())*225
    img = np.uint8(img)
    image = Image.fromarray(img, mode='CMYK')
    image = convert_cmyk_to_rgb(image)

    prefix = str(n)
    prefix_size = len(prefix)

    filename = "".join(["0" for j in range( 5 - prefix_size )] + [str(n)] )

    print(filename)
    image.save(f"cmyk_xyz_rotation_{neurons}_{layers}/{filename}.jpeg")  

00000
00001
00002
00003
00004
00005
00006
00007
00008
00009
00010
00011
00012
00013
00014
00015
00016
00017
00018
00019
00020
00021
00022
00023
00024
00025
00026
00027
00028
00029
00030
00031
00032
00033
00034
00035
00036
00037
00038
00039
00040
00041
00042
00043
00044
00045
00046
00047
00048
00049
00050
00051
00052
00053
00054
00055
00056
00057
00058
00059
00060
00061
00062
00063
00064
00065
00066
00067
00068
00069
00070
00071
00072
00073
00074
00075
00076
00077
00078
00079
00080
00081
00082
00083
00084
00085
00086
00087
00088
00089
00090
00091
00092
00093
00094
00095
00096
00097
00098
00099
00100
00101
00102
00103
00104
00105
00106
00107
00108
00109
00110
00111
00112
00113
00114
00115
00116
00117
00118
00119
00120
00121
00122
00123
00124
00125
00126
00127
00128
00129
00130
00131
00132
00133
00134
00135
00136
00137
00138
00139
00140
00141
00142
00143
00144
00145
00146
00147
00148
00149
00150
00151
00152
00153
00154
00155
00156
00157
00158
00159
00160
00161
00162
00163
00164
00165
0016

In [159]:
torch.save(model, f"saved_models/{neurons}_{layers}_noise.pt")

## Full automation

In [13]:
from time import time

In [7]:
def run_ffmepeg(folder):

    ffmpeg_path = r"C:\\ffmpeg\\bin\\ffmpeg.exe"  # Change this to your actual path

    cwd_path = rf"C:\\Users\\anthony\\OneDrive\Documentos\\random_network\\full_automation\\{folder}"

    ffmpeg_command = [
        ffmpeg_path,
        "-framerate", "30",
        "-i", "%05d.jpeg",
        "-c:v", "libx264",
        "-crf", "23",
        "-preset", "slow",
        "-pix_fmt", "yuv420p",
        "output.mp4"
    ]
    res = subprocess.run(ffmpeg_command, cwd=cwd_path, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    # print("STDOUT:\n", res.stdout)
    # print("STDERR:\n", res.stderr) 

In [ ]:
os.makedirs(f"full_automation", exist_ok = True)

r = 1000
rz = 1
l = 5
x_range = (-l, l)
y_range = (-l, l)
z_range = (0, 0)
Rx = 1920*2
Ry = 1080*2
resolution = (Rx, Ry, rz)
N_frames = 30*60


for i in range(10):

    neurons = np.random.randint(20, 40)
    layers = np.random.randint(20, 40)
    step = np.random.uniform(0.0001, 1)
    step_final = np.random.uniform(0.0001, 1)

    # noise_value = np.random.randint()

    model = CMYKModel(input_size=3, N_neurons=neurons, N_layers=layers).to(device)
    os.makedirs(f"full_automation/cmyk_{neurons}_{layers}_{i}", exist_ok = True)
    torch.save(model, f"saved_models/model.pt")
    
    # set rotations
    x_rotation = 0
    y_rotation = 0
    z_rotation = 0
    is_rotating = np.random.randint(0,2)
    if is_rotating==1:
        x_rotation = np.random.randint(0,2)
        y_rotation = np.random.randint(0,2)
        y_rotation = np.random.randint(0,2)
    
    # set zooming
    is_zooming =  np.random.randint(0,2)
    if is_zooming==1:
        step_weight = (step_final/step)**(1/1800)
    
    # set translation
    translation_vector = torch.tensor([0,0,0], dtype=torch.float, device=device)
    x_translation = 0
    y_translation = 0
    z_translation = 0
    is_translating =  np.random.randint(0,2)
    if is_translating==1:
        x_translation = np.random.uniform(0,2)*np.random.randint(0,2)
        y_translation = np.random.uniform(0,2)*np.random.randint(0,2)
        z_translation = np.random.uniform(0,2)*np.random.randint(0,2)
    translation_step = torch.tensor([x_translation,y_translation,z_translation], dtype=torch.float, device=device)
    

    # set noise
    # is_noise = np.random.randint(0,2)
    # if is_noise == 1:
    #     noise_weight = (0.3/0.001)**(1/1800)
        
    print(f"NNet: N{neurons}, L{layers}, Rotation: {x_rotation},{y_rotation},{z_rotation}, Translation: {x_translation}, {y_translation}, {z_translation}, Step, {step},{step_final}")

    start_time = time()
    for n, t in enumerate(np.linspace(0, 2*np.pi, N_frames)):

        # update parameters if needed
        if is_zooming==1:
            step *= step_weight
        if is_translating:
            translation_vector += translation_step

        # apply transformations
        x = create_3d_grid(x_range, y_range, z_range, resolution, step)
        x = torch.tensor(x, dtype=torch.float, device=device)
        x = rotate_3d_tensor(x, x_rotation*t, y_rotation*t, z_rotation*t)
        x = translate_3d_tensor(x, translation_vector)

        # get output
        output = model.forward(x).cpu()
        output = output.reshape((Ry,Rx,rz,4))

        # process image
        img = output[:,:,:]
        img = (img - img.min()) / (img.max() - img.min())*225
        img = np.uint8(img)
        image = Image.fromarray(img, mode='CMYK')
        image = convert_cmyk_to_rgb(image)

        # save image
        prefix = str(n)
        prefix_size = len(prefix)

        filename = "".join(["0" for j in range( 5 - prefix_size )] + [str(n)] )

        # print(filename)
        image.save(f"full_automation/cmyk_{neurons}_{layers}_{i}/{filename}.jpeg")

        print(f"Time per frame {(time()-start_time) / (n+1)}") 

    print('starting ffmpeg')
    run_ffmepeg(f"cmyk_{neurons}_{layers}_{i}") 
    print('done ffmpeg')



NNet: N44, L39, Rotation: 1,0,0, Translation: 0, 0, 0, Step, 1.842826977089665,1.3254415790679628
Time per frame 3.2059690952301025
Time per frame 3.2095001935958862
Time per frame 3.196656068166097
Time per frame 3.1912420988082886
Time per frame 3.186600160598755
Time per frame 3.1910003423690796
Time per frame 3.1905714103153775
Time per frame 3.1937460601329803
Time per frame 3.1945521301693387
Time per frame 3.196200060844421
Time per frame 3.197633461518721
Time per frame 3.1972500681877136
Time per frame 3.197843680014977
Time per frame 3.1985000201634
Time per frame 3.1983312129974366
Time per frame 3.1982500106096268
Time per frame 3.199056961957146
Time per frame 3.199998233053419
Time per frame 3.1999470811141166
Time per frame 3.19999840259552
Time per frame 3.198999983923776
Time per frame 3.196180582046509
Time per frame 3.193521727686343
Time per frame 3.1942904790242515
Time per frame 3.1945199871063235
Time per frame 3.1961527604323168
Time per frame 3.1970584745760315

KeyboardInterrupt: 

## Model curation

In [206]:
r = 1000
rz = 1
l = 5
x_range = (-l, l)
y_range = (-l, l)
z_range = (0, 0)
Rx = 1920//2
Ry = 1080//2
resolution = (Rx, Ry, rz)

neurons = 30
layers = 40

images = []

os.makedirs(f"curate_images", exist_ok = True)

step=0.05
x = create_3d_grid(x_range, y_range, z_range, resolution, step)
x_r = torch.tensor(x, dtype=torch.float, device=device)

for neurons in range(50,100,20):
    for layers in range(50,100,20):
        for n in range(100):

            model = CMYKModel(input_size=3, N_neurons=neurons, N_layers=layers).to(device)
            
            output = model.forward(x_r).cpu()
            output = output.reshape((Ry,Rx,rz,4))

            img = output[:,:,:]

            img = (img - img.min()) / (img.max() - img.min())*225
            img = np.uint8(img)
            image = Image.fromarray(img, mode='CMYK')
            image = convert_cmyk_to_rgb(image)

            prefix = str(n)
            prefix_size = len(prefix)

            filename = "".join(["0" for j in range( 5 - prefix_size )] + [str(n)] )

            print(filename)
            image.save(f"curate_images/curated_{neurons}_{layers}_{filename}.jpeg")  
            torch.save(model, f"saved_models/curated_{neurons}_{layers}_{filename}.pt")

00000
00001
00002
00003
00004
00005
00006
00007
00008
00009
00010
00011
00012
00013
00014
00015
00016
00017
00018
00019
00020
00021
00022
00023
00024
00025
00026
00027
00028
00029
00030
00031
00032
00033
00034
00035
00036
00037
00038
00039
00040
00041
00042
00043
00044
00045
00046
00047
00048
00049
00050
00051
00052
00053
00054
00055
00056
00057
00058
00059
00060
00061
00062
00063
00064
00065
00066
00067
00068
00069
00070
00071
00072
00073
00074
00075
00076
00077
00078
00079
00080
00081
00082
00083
00084
00085
00086
00087
00088
00089
00090
00091
00092
00093
00094
00095
00096
00097
00098
00099
00000
00001
00002
00003
00004
00005
00006
00007
00008
00009
00010
00011
00012
00013
00014
00015
00016
00017
00018
00019
00020
00021
00022
00023
00024
00025
00026
00027
00028
00029
00030
00031
00032
00033
00034
00035
00036
00037
00038
00039
00040
00041
00042


KeyboardInterrupt: 

## Portrait space

In [ ]:
r = 1000
rz = 1
l = 5
x_range = (-l, l)
y_range = (-l, l)
z_range = (0, 0)
Rx = 250
Ry = 250
resolution = (Rx, Ry, rz)

images = []

# torch.save(model, "saved_models/model.pt")

step=0.1
x_r = create_3d_grid(x_range, y_range, z_range, resolution, step)
x_r = torch.tensor(x_r, dtype=torch.float, device='cuda')

os.makedirs(f"portrait2", exist_ok = True)


for L in range(20, 75, 5):
    for N in range(20, 75, 5):

        os.makedirs(f"portrait2/{N}_{L}", exist_ok = True)
        model = CMYKModel(input_size=3, N_neurons=N, N_layers=L).to(device)

        for n,t in enumerate(np.linspace(0, 2*np.pi, 30*60)):


            x_r = create_3d_grid(x_range, y_range, z_range, resolution, step)
            x_r = torch.tensor(x_r, dtype=torch.float, device='cuda')
            x_r = rotate_3d_tensor(x_r, 0, t, 0)


            output = model.forward(x_r).cpu()
            output = output.reshape((Ry,Rx,rz,4))

            img = output[:,:,:]

            img = (img - img.min()) / (img.max() - img.min())*225
            img = np.uint8(img)
            image = Image.fromarray(img, mode='CMYK')

            image = convert_cmyk_to_rgb(image)

            prefix = str(n)
            prefix_size = len(prefix)

            filename = "".join(["0" for j in range( 5 - prefix_size )] + [str(n)] )

            print(filename)
            image.save(f"portrait2/{N}_{L}/{filename}.jpeg")  


        

00000
00001
00002
00003
00004
00005
00006
00007
00008
00009
00010
00011
00012
00013
00014
00015
00016
00017
00018
00019
00020
00021
00022
00023
00024
00025
00026
00027
00028
00029
00030
00031
00032
00033
00034
00035
00036
00037
00038
00039
00040
00041
00042
00043
00044
00045
00046
00047
00048
00049
00050
00051
00052
00053
00054
00055
00056
00057
00058
00059
00060
00061
00062
00063
00064
00065
00066
00067
00068
00069
00070
00071
00072
00073
00074
00075
00076
00077
00078
00079
00080
00081
00082
00083
00084
00085
00086
00087
00088
00089
00090
00091
00092
00093
00094
00095
00096
00097
00098
00099
00100
00101
00102
00103
00104
00105
00106
00107
00108
00109
00110
00111
00112
00113
00114
00115
00116
00117
00118
00119
00120
00121
00122
00123
00124
00125
00126
00127
00128
00129
00130
00131
00132
00133
00134
00135
00136
00137
00138
00139
00140
00141
00142
00143
00144
00145
00146
00147
00148
00149
00150
00151
00152
00153
00154
00155
00156
00157
00158
00159
00160
00161
00162
00163
00164
00165
0016

KeyboardInterrupt: 

In [ ]:

x= torch.tensor(x, dtype=torch.float, device=device)

x = rotate_3d_tensor(x, 0.3, 0, 0)

output = model.forward(x).cpu()
output = output.reshape((r,r,rz,4))

img = output[:,:,:]

img = (img - img.min()) / (img.max() - img.min())*225
img = np.uint8(img)
image = Image.fromarray(img, mode='CMYK')

draw = ImageDraw.Draw(image)
# draw.rectangle(xy=(foo, bar, essential_value_one, essential_value_two))

imgBytes = BytesIO()
image.save(imgBytes, format="jpeg")

ipywidgets.Image(value=imgBytes.getvalue())

C:\Users\anthony\AppData\Local\Temp\2\ipykernel_9152\3563829209.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x= torch.tensor(x, dtype=torch.float, device=device)


Image(value=b'\xff\xd8\xff\xee\x00\x0eAdobe\x00d\x00\x00\x00\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\…